In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import torch
from torch import nn

from transformers import AutoModel
from model.layer import CharCNN, FeatureEmbedding
from model.layer.utils import get_extended_attention_mask
from model.layer.featureEmbed import FeatureRep

In [3]:
# bert_layers = nn.ModuleList([AutoModel.from_pretrained('vinai/phobert-base').base_model.encoder.layer[i] for i in range(2)])

In [4]:
# print(sum([i.numel() for i in bert_layers.parameters()]))

In [5]:
# bert_embed = AutoModel.from_pretrained('vinai/phobert-base').base_model.embeddings

In [6]:
# print(sum([i.numel() for i in bert_embed.parameters()]))

In [7]:
# model = nn.Sequential(
#     BertTop(top=2),
#     nn.LSTM(768, 768//2, batch_first=True, num_layers=2,bidirectional=True),
    
    
# )

In [8]:
from preprocess.static_features import FeatureExtractor, Feature,NERdataset,logger, init_logger
from preprocess.processcer_join_bert import NERProcessor

In [9]:
 feats=Feature("/home/tuenguyen/Desktop/24mar2021/task_nlp/join_task_gender_department/resources/features/feature_config.json")

In [10]:
# list(model.word_net.parameters())

In [11]:
# model.word_net.feature_embeddings
# a[0]

In [12]:
import torch
from transformers import AutoModel, AutoTokenizer
# For transformers v4.x+: 
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
processor = NERProcessor("./dataset", tokenizer)
processor.labels=["O",'B-GENDER','I-GENDER','B-LOC','I-LOC']
processor.label_map= {label: i for i, label in enumerate(processor.labels, 1)}
a = processor.get_example("train",use_feats=True)

# features = processor.convert_examples_to_features(a, 126,feats)
# print(features)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [13]:
import os
import argparse
import random
import numpy as np

from torch.utils.data import DataLoader, RandomSampler
from torch.utils.tensorboard import SummaryWriter

from tqdm import tqdm
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers.tokenization_bert import BertTokenizer
from sklearn.metrics import classification_report, f1_score


def build_dataset(args, processor, data_type='train', feature=None, device=torch.device('cpu')):
    # Load data features from cache or dataset file
    cached_features_file = os.path.join(args['data_dir'], 'cached_{}_{}_{}'.format(
        data_type,
        list(filter(None, args['model_name_or_path'].split('/'))).pop(),
        str(args['max_seq_length'])))

    if os.path.exists(cached_features_file):
        print("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        print("Creating features from dataset file at %s", args['data_dir'])
        examples = processor.get_example(data_type, feature is not None)
        features = processor.convert_examples_to_features(examples, args['max_seq_length'], feature)
        print("Saving features into cached file %s", cached_features_file)
        torch.save(features[:-1], cached_features_file)
    return NERdataset(features, device)



def caculator_metric(preds, golds, labels):
#     pred_iob_labels = [labels[label_id] for label_id in preds]
#     gold_iob_labels = [labels[label_id] for label_id in golds]

#     pred_labels = [labels[label_id - 1].split("-")[-1].strip() for label_id in preds]
#     gold_labels = [labels[label_id - 1].split("-")[-1].strip() for label_id in golds]

#     iob_metric = classification_report(pred_iob_labels, gold_iob_labels, output_dict=True)
    metric = classification_report(preds, golds, output_dict=True)

    return metric, metric


def update_model_weights(model, iterator, optimizer, scheduler):
    # init static variables
    tr_loss = 0
    model.train()

    for step, batch in enumerate(tqdm(iterator, desc="Iteration")):
        optimizer.zero_grad()
        tokens, token_ids, attention_masks, token_mask, segment_ids, label_ids, label_masks, feats = batch
        loss, _ = model.calculate_loss(token_ids, attention_masks, token_mask, segment_ids, label_ids, label_masks,
                                       feats)
        tr_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.)
        
        optimizer.step()
        scheduler.step()
        
    return tr_loss


def evaluate(model, iterator, label_map):
    # init static variables
    preds = []
    golds = []
    eval_loss = 0
    model.eval()

    for step, batch in enumerate(tqdm(iterator, desc="Iteration")):
        tokens, token_ids, attention_masks, token_mask, segment_ids, label_ids, label_masks, feats = batch
        loss, (logits, labels) = model.calculate_loss(token_ids, attention_masks, token_mask, segment_ids, label_ids,
                                                      label_masks, feats)
        eval_loss += loss.item()
        logits = torch.argmax(nn.functional.softmax(logits, dim=-1), dim=-1)
        pred = logits.detach().cpu().numpy()
        gold = labels.to('cpu').numpy()
        preds.extend(pred)
        golds.extend(gold)
#     print(preds,golds)
    preds = np.concatenate(preds)
    golds=np.concatenate(golds)
    iob_metric, metric = caculator_metric(preds, golds, label_map)

    return eval_loss, iob_metric, metric




In [14]:
a=np.random.randn(32,)
b=a
np.concatenate([a,b]).shape

(64,)

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
random.seed(23072021)
np.random.seed(23072021)
torch.manual_seed(23072021)
summary_writer = SummaryWriter('./logs')
init_logger(f"./logs/vner_trainning.log")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
processor = NERProcessor("./dataset", tokenizer)
processor.labels=["O",'B-GENDER','I-GENDER','B-LOC','I-LOC']
processor.label_map= {label: i for i, label in enumerate(processor.labels, 1)}
num_labels = processor.get_num_labels()
logger.info("Build data ...")
args={
    'model_name_or_path':'phobert',
    'data_dir':'./dataset',
    'max_seq_length':100
}
train_data = build_dataset(args, processor, data_type='train', feature=feats, device=device)
eval_data = build_dataset(args, processor, data_type='test', feature=feats, device=device)

train_sampler = RandomSampler(train_data)
train_iterator = DataLoader(train_data, sampler=train_sampler, batch_size=64)

eval_sampler = RandomSampler(eval_data)
eval_iterator = DataLoader(eval_data, sampler=eval_sampler, batch_size=32)


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Build data ...


Loading features from cached file %s ./dataset/cached_train_phobert_100
Loading features from cached file %s ./dataset/cached_test_phobert_100


In [16]:
a=next(iter(train_iterator))

In [17]:
a[1]

tensor([[    3,  6207, 20827,  ...,     0,     0,     0],
        [    3,   168,  9073,  ...,     0,     0,     0],
        [    3,  2080,    33,  ...,     0,     0,     0],
        ...,
        [    3,  1144,   343,  ...,     0,     0,     0],
        [    3,   793,  2900,  ...,     0,     0,     0],
        [    3,  5217,    41,  ...,     0,     0,     0]], device='cuda:0')

In [18]:
a[2],a[5]

(tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'),
 tensor([[0, 1, 1,  ..., 0, 0, 0],
         [0, 1, 1,  ..., 0, 0, 0],
         [0, 1, 1,  ..., 0, 0, 0],
         ...,
         [0, 1, 1,  ..., 0, 0, 0],
         [0, 1, 1,  ..., 0, 0, 0],
         [0, 1, 1,  ..., 0, 0, 0]], device='cuda:0'))

In [19]:
# sample.tokens, token_id_tensors, attention_mask_tensors, token_mask_tensors, segment_id_tensors, \
#                label_id_tensors, label_mask_tensors, feat_tensors

In [20]:
# input_ids, attention_masks, token_masks, segment_ids, label_ids, label_masks, feats

In [16]:
from model.model import BertCRF

In [17]:
model = BertCRF(n_layer_bert=2)
model = model.to("cuda")
model.load_state_dict(torch.load("/home/tuenguyen/Desktop/24mar2021/task_nlp/join_task_gender_department/checkpoints/vner_model.bin"))

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


<All keys matched successfully>

In [18]:
model.train()
best_score = -1
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay': 1e-4},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
num_train_optimization_steps = len(train_iterator) // 1 * 100
warmup_steps = int(0.1 * num_train_optimization_steps)
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=3e-4,)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps,
                                            num_training_steps=num_train_optimization_steps)
for e in range(2):
    logger.info("="*30 + f"Epoch {e}" + "="*30)
    tr_loss = update_model_weights(model, train_iterator, optimizer, scheduler)
    logger.info(f"train Loss: {tr_loss}")
    eval_loss, iob_metric, metric = evaluate(model, eval_iterator, processor.labels)
    logger.info(f"eval Loss: {eval_loss}")
    logger.info(f"F1-Score tag: {metric['macro avg']['f1-score']}")
    logger.info(f"F1-Score IOB-tag: {iob_metric['macro avg']['f1-score']}")
    logger.info(f"Metric:")
#     print(metric)
    logger.info(f"\tO: {metric['1']['f1-score']}")
    logger.info(f"\tB-GENDER: {metric['2']['f1-score']} ")
    logger.info(f"\tI-GENDER: {metric['3']['f1-score']}")
    logger.info(f"\tB-LOC: {metric['4']['f1-score']}")
    logger.info(f"\tI-LOC: {metric['5']['f1-score']}")


    if iob_metric['macro avg']['f1-score'] > best_score:
        best_score = iob_metric['macro avg']['f1-score']
        best_epoch = e
        model_path = f"checkpoints/vner_model.bin"
        torch.save(model.state_dict(), model_path)
        logger.info(f"Model save at epoch {best_epoch} with best score {best_score}")
        summary_writer.add_text("Best result", f"F1-Score: {best_score}", best_epoch)
        summary_writer.flush()


==============================Epoch 0==============================
Iteration: 100%|██████████| 125/125 [02:19<00:00,  1.12s/it]
train Loss: 9.353205114603043
Iteration: 100%|██████████| 70/70 [00:15<00:00,  4.66it/s]
eval Loss: 87.12612441182137
F1-Score tag: 0.8740105381527828
F1-Score IOB-tag: 0.8740105381527828
Metric:
	O: 0.9966328974028366
	B-GENDER: 0.9370134465675866 
	I-GENDER: 0.5
	B-LOC: 0.8931419457735247
	I-LOC: 0.9172749391727495
Model save at epoch 0 with best score 0.8740105381527828
==============================Epoch 1==============================
Iteration: 100%|██████████| 125/125 [02:12<00:00,  1.06s/it]
train Loss: 4.99702849262394
Iteration: 100%|██████████| 70/70 [00:14<00:00,  4.77it/s]
eval Loss: 99.12007230520248
F1-Score tag: 0.8735246318054665
F1-Score IOB-tag: 0.8735246318054665
Metric:
	O: 0.996729329954623
	B-GENDER: 0.9405940594059405 
	I-GENDER: 0.4878048780487805
	B-LOC: 0.8951386098834874
	I-LOC: 0.9208809135399674


In [24]:
 processor.labels

['O', 'B-GENDER', 'I-GENDER', 'B-LOC', 'I-LOC']

In [25]:
text = 'em muốn đặt khám cho mẹ em thái tự mẫu'
from underthesea import word_tokenize

In [20]:
text = 'cho anh nguyễn   văn nam ~  xin thông tin khám bệnh ở khoa cấp cứu phía miền nam'
predict, sentence=model.predict(text,device='cuda')
print(list(zip(sentence.split(),predict)))

100%|██████████| 1/1 [00:00<00:00, 860.02it/s]

[('cho', 'O'), ('anh', 'B-GENDER'), ('nguyễn_văn', 'O'), ('nam', 'O'), ('~', 'O'), ('xin', 'O'), ('thông_tin', 'O'), ('khám', 'O'), ('bệnh', 'O'), ('ở', 'O'), ('khoa', 'B-LOC'), ('cấp_cứu', 'I-LOC'), ('phía', 'O'), ('miền', 'O'), ('nam', 'O')]


In [24]:
len(sentence),len(predict)

(15, 15)

In [31]:
# sentence= sentence.split()
entities = predict
entities_final = []
step = 0
while step < len(sentence):
    if entities[step] == 'O':
        entities_final.append([entities[step],sentence[step]])
        step += 1
    elif 'B' in entities[step]:
        a = entities[step].split("-")[1]
        entities_final.append([a, ""])
        while step < len(sentence) and "-" in entities[step] and entities[step].split("-")[1] == a:
            entities_final[-1][1] = entities_final[-1][1] + " " + sentence[step]
            step += 1


In [32]:
entities_final

[['O', 'cho'],
 ['GENDER', ' anh'],
 ['O', 'nguyễn_văn'],
 ['O', 'nam'],
 ['O', '~'],
 ['O', 'xin'],
 ['O', 'thông_tin'],
 ['O', 'khám'],
 ['O', 'bệnh'],
 ['O', 'ở'],
 ['LOC', ' khoa cấp_cứu'],
 ['O', 'phía'],
 ['O', 'miền'],
 ['O', 'nam']]

In [33]:
from preprocess.matcher_lib import GenderMatcher, DepartmentMatcher

{'entities': [{'start': 32, 'end': 40, 'value': 'department', 'extractor': 'template_matching_department', 'confidence': 1.0}]}
tim mạch


In [37]:
gen = DepartmentMatcher()

In [38]:
for a in entities_final:
    print(gen.match(a[1]))

{'entities': []}
{'entities': []}
{'entities': []}
{'entities': []}
{'entities': []}
{'entities': []}
{'entities': []}
{'entities': []}
{'entities': []}
{'entities': []}
{'entities': []}
{'entities': []}
{'entities': []}
{'entities': []}


In [ ]:
tokens, token_ids, attention_masks, token_mask, segment_ids, label_ids, label_masks, feats = dataset_test[0]
feats = {k:v[None,...] for k,v in feats.items()}
model.eval()
with torch.no_grad():
    out = model(token_ids[None,...], attention_masks[None,...], feats)
    out1 = model.crf.decode(out)
    out2 = torch.argmax(out,axis=-1)[0]

In [ ]:
out2.shape

In [ ]:
out2[token_mask==1]

In [ ]:
token_mask==1

In [ ]:
# out[0]

In [ ]:
out1 = [max(i-1,0) for i in out1[0]]
# out = out - 1


In [ ]:
# processor.label_map
# print([processor.labels[i] for i in out1] )
print([(ix,processor.labels[max(i-1,0)]) for i,ix in zip(out[0][1:],text.split())] )

In [ ]:
# slot_preds = np.array(model.crf.decode(out))

In [ ]:
print([processor.labels[i] for i in out1] )

In [ ]:
processor.label_map

In [ ]:
train_data[132][0],train_data[132][5]

In [ ]:
" ".join(eval_data[1000][0])

In [ ]:
from underthesea import word_tokenize

In [ ]:
text = 'cho anh nguyễn   văn nam ~  xin thông tin khám bệnh ở khoa cấp cứu phía miền nam'
from underthesea import word_tokenize
text=word_tokenize(text,format='text')
text